## Production Amounts and Scope 1 and 2 Emission Values Per EAF Plant in the United States

### Loading Libraries

In [25]:
import pandas as pd
import numpy as np
import janitor
import geopandas as gpd
import mapclassify as mc
from shapely.geometry import Point
import re

pd.options.display.float_format = '{:.5f}'.format

### Importing eGrid Data

The EPA eGrida Data, which was available for 2021 at the time of download, contains the emissions intensity of electricity produced in each sub region. For this analysis, we assume that each steel plant is drawing 100% of its electricity from the grid (and the eGrid subregion that it is located within).

In [2]:
## Importing grid region electricity values
egrid2021_data = pd.read_excel('../data/eGRID2021_data.xlsx', sheet_name = "SRL21").clean_names(strip_underscores = True).drop(0)

### Standardizing state names and cleaning up rows in eGrid data

In [3]:
## Cleaning egrid data for usability
emissions_by_subregion = egrid2021_data.copy()

## Data Processing on columns
emissions_by_subregion["co2e_lbs_per_mwh"] = pd.to_numeric(emissions_by_subregion["egrid_subregion_annual_co2_equivalent_combustion_output_emission_rate_lb_mwh"])
emissions_by_subregion['subregion'] = emissions_by_subregion['egrid_subregion_acronym']
emissions_by_subregion['subregion_name'] = emissions_by_subregion['egrid_subregion_name']
emissions_by_subregion['co2e_tonnes_per_mwh'] = emissions_by_subregion['co2e_lbs_per_mwh'] / 2204.62262185

## Filtering subregions
excluded_subregions = ["AKGD", "AKMS", "HIMS", "HIOA", "PRMS"]
emissions_by_subregion = emissions_by_subregion[~emissions_by_subregion['subregion'].isin(excluded_subregions)]

## Selecting desired columns
emissions_by_subregion = emissions_by_subregion[['subregion', 'subregion_name', 'co2e_tonnes_per_mwh']]

### Importing GEM data

In [4]:
gem_data_readin = pd.read_excel("../data/GEM_2022_data.xlsx", sheet_name = "Steel Plants").clean_names(strip_underscores = True)

### Filtering to only look at plants and data that we are interested in

In [5]:
## eaf_capacity is in thousand tonnes per year and we are standardizing to tonnes
gem_data_cleaned = gem_data_readin.copy()

## Filtering to the specifications we need
gem_data_cleaned = gem_data_cleaned[
       (gem_data_cleaned['country'] == "United States") &
       (gem_data_cleaned["status"] == "operating") &
       gem_data_cleaned['nominal_eaf_steel_capacity_ttpa'].notna()
]

## Needed to do this in another step to make sure start_date was properly filtered
gem_data_cleaned = gem_data_cleaned[
       (gem_data_cleaned['start_date'] < 2022) &
       (~gem_data_cleaned['plant_id'].isin(["SUS00009", "SUS00061"]))
]

## Renaming columns
gem_data_cleaned = gem_data_cleaned.rename(columns={'plant_name_english':'plant_name'
                                          , 'subnational_unit_province_state':'state'
                                          , 'location_address':'address'})

## Converting EAF capacity from Thousand Tonnes to Tonnes
gem_data_cleaned['eaf_capacity'] = pd.to_numeric(gem_data_cleaned['nominal_eaf_steel_capacity_ttpa'])
gem_data_cleaned['max_tonnes_of_steel_producible_annually'] = gem_data_cleaned['eaf_capacity'] * 1000

## Reordering columns to desired order
gem_data_cleaned = gem_data_cleaned[['plant_id'
         , 'plant_name'
         , 'owner'
         , 'coordinates'
         , 'country'
         , 'state'
         , 'status'
         , 'start_date'
         , 'plant_age_years'
         , 'max_tonnes_of_steel_producible_annually'
         , 'municipality'
         , 'address'
         , 'category_steel_product'
         , 'steel_products'
         , 'responsiblesteel_certification']]

## Removing columns we do not need
gem_data = gem_data_cleaned.drop(columns=['country', 'start_date', 'status', 'responsiblesteel_certification'])

## Separate the "coordinates" column into "lat" and "lon" columns
gem_data[['lat', 'lon']] = gem_data['coordinates'].str.split(',', expand=True)

## Remove the "coordinates" column
gem_data.drop(columns=['coordinates'], inplace=True)

## Reordering columns
gem_data = gem_data[['plant_id', 'plant_name', 'owner', 'lat', 'lon', 'state', 'plant_age_years', 'max_tonnes_of_steel_producible_annually', 'municipality', 'address', 'category_steel_product', 'steel_products']]

In [6]:
## Reading in data
subregion_shapes_raw = gpd.read_file("../data/egrid2020_subregions/eGRID2020_subregions.shp").clean_names()

## Filtering subregion shapes
subregion_shapes = subregion_shapes_raw[~subregion_shapes_raw['zipsubregi'].isin(["AKGD", "AKMS", "HIMS", "HIOA", "PRMS"])]

## Simplifying subregion shapes
subregion_shapes['geometry'] = subregion_shapes.simplify(tolerance=0.0005)

## Bringing in plant points
plant_points = gpd.GeoDataFrame(gem_data, geometry=gpd.points_from_xy(gem_data['lon'], gem_data['lat']))
plant_points.crs = "EPSG:4326"

## Finding points that overlap with the egrid subregion
## Removing columns we dont need and renaming the subregion column
plant_emissions_by_subregion = gpd.sjoin(plant_points, subregion_shapes, op='within').drop(columns=['geometry', 'index_right', 'shape_leng', 'shape_le_1', 'shape_area']).rename(columns={'zipsubregi':'subregion'})

plant_emissions_by_subregion = plant_emissions_by_subregion.merge(emissions_by_subregion, on='subregion', how='left')

/Users/mikey/opt/miniconda3/envs/mikeys_env/lib/python3.10/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


### Importing AISI data

In [7]:
AISI_regions_readin = pd.read_excel("../data/AISI_regions.xlsx", sheet_name="Regions by State").clean_names(strip_underscores = True)

AISI_data_readin = pd.read_excel("../data/AISI_data.xlsx", sheet_name="AISI Production Values").clean_names(strip_underscores = True)

NE = Northeast

GL = Great Lakes

MW = Midwest

S = Southern

W = Western

### Filtering AISI data, renaming columns, and selecting the weekly data and utilization rates we need

**Uilization is based on tonnage capability to produce raw steel for a sustained full order book.**

In [8]:
AISI_regions = AISI_regions_readin[AISI_regions_readin['steel_plant_in_gspt'] == "yes"]

AISI_data = AISI_data_readin.rename(columns = {'north_east_region_capacity_utilization':'NE_util'
                                          , 'great_lakes_region_capacity_utilization':'GL_util'
                                          , 'midwest_region_capacity_utilization':'MW_util'
                                          , 'southern_region_capacity_utilization':'S_util'
                                          , 'western_region_capacity_utilization':'W_util'})

AISI_data = AISI_data[['week_end_date', 'NE_util', 'GL_util', 'MW_util', 'S_util', 'W_util']]

AISI_data = AISI_data[AISI_data['week_end_date'] <= '2022-01-01']

AISI_data['week_end_date'] = pd.to_datetime(AISI_data['week_end_date'])

AISI_data['week_end_date'] = AISI_data['week_end_date'].dt.date

### Merging all of our data so far

In [9]:
regional_plant_emissions = pd.merge(plant_emissions_by_subregion, AISI_regions, on = "state", how = "left")

regional_plant_emissions = regional_plant_emissions.drop(['steel_plant_in_gspt', 'state_abbreviation'], axis = 1)

conditions = [
    regional_plant_emissions['municipality'].isin(["Alton", "Sterling", "Peoria", "Granite City", "Mansfield", "Middletown"])
    , regional_plant_emissions['municipality'].isin(["Riverdale", "Chicago", "Bourbonnais", "Cuyahoga Heights", "Cleveland", "Toledo", "Lorain"])
    , regional_plant_emissions['municipality'].isin(["Mingo Junction", "Youngstown", "Canton"])
]

choices = ["Midwest", "Great Lakes", "North East"]

regional_plant_emissions['region'] = np.select(conditions, choices, default=regional_plant_emissions['region'])

### Defining carbon intensity for eaf steel production and preparing data to calculate intensities

In [10]:
## co2e_tonnes_per_mwh is from eGrid and is pounds of CO2e per MWH of electricity produced per grid location (not regional location)

## Global Efficiency Intelligence states that it takes 710 KWH to produce 1 tonne of steel. "Global Efficiency Intelligence: Industrial Electrification in U.S. States"
## MWH per tonne of steel
eaf_MWH_per_tonne = 710 / 1000

## emissions_intensity is tonnes of CO2e per tonne of steel (the amount of co2e produced for every tonne of steel produced)

AISI_longer = AISI_data.melt(id_vars= 'week_end_date'
                           , value_vars = ['NE_util', 'GL_util', 'MW_util', 'S_util', 'W_util']
                           , var_name = 'region'
                           , value_name = 'utilization')

region_conditions = [
  AISI_longer['region'] == "NE_util"
  , AISI_longer['region'] == "GL_util"
  , AISI_longer['region'] == "MW_util"
  , AISI_longer['region'] == "S_util"
  , AISI_longer['region'] == "W_util"
]

region_choices = ["North East", "Great Lakes", "Midwest", "Southern", "Western"]

AISI_longer['region'] = np.select(region_conditions, region_choices)

### Finding Weekly Scope 2 co2e values and putting it in long and wide format

In [16]:
## Merging our two datasets based on the region column and defining new columns
scope2_plant_emissions_long = pd.merge(regional_plant_emissions, AISI_longer, how = "left", on = "region")

## Creating relavent columns
scope2_plant_emissions_long['estimated_emissions_intensity_tonne_per_tonne_scope2'] = scope2_plant_emissions_long['co2e_tonnes_per_mwh'] * eaf_MWH_per_tonne
scope2_plant_emissions_long['max_tonnes_of_steel_producible_weekly'] = scope2_plant_emissions_long['max_tonnes_of_steel_producible_annually'] / 52
scope2_plant_emissions_long['scope2_co2e_tonnes_per_week'] = scope2_plant_emissions_long['utilization'] * scope2_plant_emissions_long['max_tonnes_of_steel_producible_weekly']

scope2_plant_emissions_long = scope2_plant_emissions_long[["plant_id"
                                                            ,"plant_name"                                          
                                                            ,"owner"                                               
                                                            ,"lat"                                                 
                                                            ,"lon"                                                 
                                                            ,"state"                                               
                                                            ,"plant_age_years"                                     
                                                            ,"municipality"                                        
                                                            ,"address"                                             
                                                            ,"category_steel_product"                              
                                                            ,"steel_products"                                      
                                                            ,"subregion"                                           
                                                            ,"subregion_name"                                      
                                                            ,"co2e_tonnes_per_mwh"                                 
                                                            ,"region"                                              
                                                            ,"week_end_date"                                       
                                                            ,"utilization"                                         
                                                            ,"estimated_emissions_intensity_tonne_per_tonne_scope2"
                                                            ,"max_tonnes_of_steel_producible_annually"             
                                                            ,"max_tonnes_of_steel_producible_weekly"               
                                                            ,"scope2_co2e_tonnes_per_week"]]

## Rounding the data
scope2_plant_emissions_long_rounded = scope2_plant_emissions_long.round(2)

## Transferring each week to be its own column
scope2_plant_emissions_wide = scope2_plant_emissions_long_rounded.drop(columns = ['utilization']).pivot(
    index = ['plant_id', 'plant_name', 'owner', 'lat', 'lon', 'state',
       'plant_age_years', 'municipality', 'address', 'category_steel_product',
       'steel_products', 'subregion', 'subregion_name', 'co2e_tonnes_per_mwh',
       'region',
       'estimated_emissions_intensity_tonne_per_tonne_scope2',
       'max_tonnes_of_steel_producible_annually',
       'max_tonnes_of_steel_producible_weekly']
    , columns = 'week_end_date'
    , values = 'scope2_co2e_tonnes_per_week'
).reset_index(drop=False)


/Users/mikey/opt/miniconda3/envs/mikeys_env/lib/python3.10/site-packages/pandas/core/algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


### Finding Scope 2 values for the complete 2021 year

In [26]:
yearly_scope2 = scope2_plant_emissions_long.groupby(['plant_id', 'plant_name', 'address', 'lat', 'lon']).agg(scope2_co2e_tonnes_2021 = ('scope2_co2e_tonnes_per_week', 'sum')).reset_index()

'\nyearly_scope2 = scope2_plant_emissions_long %>% \n  group_by(plant_id, plant_name, address, lat, lon) %>% \n  summarize(scope2_co2e_tonnes_2021 = sum(scope2_co2e_tonnes_per_week)) %>% \n  ungroup()\n\n'

### Extracting zip codes to find matches with addresses

In [52]:
## Making a copt of yearly_scope2 to work with
zipcode_extraction = yearly_scope2.copy()

## Extracting zip code from address
zipcode_extraction['address2'] = yearly_scope2['address']

## Separate the addresses based off of commas
zipcode_extraction[['zip_code_part1', 'zip_code_part2', 'zip_code_part3', 'zip_code_part4', 'zip_code_part5']] = zipcode_extraction['address2'].str.split(',', expand = True)

## Drop columns we do not need
zipcode_extraction = zipcode_extraction.drop(columns = ['address2', 'zip_code_part1', 'zip_code_part5'])

## Extract only the numeric portions of the addresses and put in NA's if there are no numbers
zipcode_extraction['zip_code_part2'] = zipcode_extraction['zip_code_part2'].str.extract(r'(\d+)')
zipcode_extraction['zip_code_part3'] = zipcode_extraction['zip_code_part3'].str.extract(r'(\d+)')
zipcode_extraction['zip_code_part4'] = zipcode_extraction['zip_code_part4'].str.extract(r'(\d+)')

## Create zip_code column by selecting the non-null values from zip_code_part2, zip_code_part3, zip_code_part4
zipcode_extraction['zip_code'] = zipcode_extraction['zip_code_part2'].fillna(zipcode_extraction['zip_code_part3']).fillna(zipcode_extraction['zip_code_part4'])

## Dropping columns we do not need
zipcode_extraction = zipcode_extraction.drop(columns = ['zip_code_part2', 'zip_code_part3', 'zip_code_part4'])

checking_unique_zipcodes_scope2 = zipcode_extraction.groupby('zip_code').size().reset_index(name='count')
"""

checking_unique_zipcodes_scope2 <- zipcode_extraction %>% 
  group_by(zip_code) %>% 
  summarise(count = n()) %>% 
  filter(count < 2) %>% 
  drop_na() %>%
  ungroup() %>% 
  select(zip_code)

unique_scope2_zip_codes <- checking_unique_zipcodes_scope2 %>% 
  left_join(zipcode_extraction, by = "zip_code")

"""

'\n\nchecking_unique_zipcodes_scope2 <- zipcode_extraction %>% \n  group_by(zip_code) %>% \n  summarise(count = n()) %>% \n  filter(count < 2) %>% \n  drop_na() %>%\n  ungroup() %>% \n  select(zip_code)\n\nunique_scope2_zip_codes <- checking_unique_zipcodes_scope2 %>% \n  left_join(zipcode_extraction, by = "zip_code")\n\n'

In [53]:
checking_unique_zipcodes_scope2

,zip_code,count
0,08872,1
1,16045,1
2,17113,1
3,19230,1
4,23805,1
5,24017,1
6,27922,1
7,29033,1
8,29450,1
9,29540,1
